<a href="https://colab.research.google.com/github/Mateus13Silva/agente-emocional-produtividade/blob/main/Agente_Emocional_Inteligente_definitivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Configura a API Key do Google Gemini
import os
from google.colab import userdata
import google.generativeai as genai

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY2')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [2]:
# 🔁 Funções de Agrupamento de Emoções
GRUPOS_EMOCAO = {
    "ansioso": ["nervoso", "tenso", "preocupado"],
    "cansado": ["exausto", "fadigado", "sem energia"],
    "feliz": ["alegre", "contente", "animado"],
    "triste": ["chateado", "deprimido", "abatido"],
    "motivado": ["determinado", "focado", "inspirado"]
}

def agrupar_emocao(emocao):
    e = emocao.strip().lower()
    for grupo, termos in GRUPOS_EMOCAO.items():
        if e == grupo or e in termos:
            return grupo
    return e

In [3]:
# 💬 Respostas automáticas baseadas na emoção
def gerar_resposta_emocional(emocao):
    emocao = agrupar_emocao(emocao)
    respostas = {
        "cansado": "Tente fazer uma pausa curta e beber água. Você está indo bem.",
        "ansioso": "Respire fundo por alguns segundos. Tente focar em uma tarefa por vez.",
        "feliz": "Que ótimo! Continue nesse ritmo positivo.",
        "triste": "Lembre-se que está tudo bem não estar bem. Seja gentil consigo mesmo.",
        "motivado": "Excelente! Mantenha o foco e aproveite a produtividade."
    }
    return respostas.get(emocao, "Estou com você! Continue seguindo em frente.")


In [4]:
# Solução alternativa para armazenamento local
# Instalar pytz e matplotlib se não estiverem instalados
!pip install -q pytz matplotlib

import json
import os
from datetime import datetime
from collections import Counter
import pytz
import matplotlib.pyplot as plt
from google.colab import files

# Diretório local para armazenamento
CAMINHO_BASE = "./meu_agente_emocional/"
os.makedirs(CAMINHO_BASE, exist_ok=True)

# Função para carregar interações do arquivo local
def carregar_interacoes():
    nome_arquivo = CAMINHO_BASE + "interacoes.json"
    try:
        with open(nome_arquivo, "r") as f:
            return json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        return []

# Função para salvar interações no arquivo local
def salvar_interacoes(interacoes):
    nome_arquivo = CAMINHO_BASE + "interacoes.json"
    nome_txt = CAMINHO_BASE + "interacoes_backup.txt"
    try:
        with open(nome_arquivo, "w") as f:
            json.dump(interacoes, f, indent=4)

        # Criar backup em .txt para fácil visualização
        with open(nome_txt, "w") as f:
            for i in interacoes:
                f.write(f"{i['timestamp']} | {i['tarefa']} | {i['emocao']} | {i['resposta']}\n")

        print("💾 Interações salvas localmente com sucesso!")
        try:
            files.download(nome_arquivo)
            print("📥 Arquivo JSON disponível para download")
            files.download(nome_txt)
            print("📥 Arquivo TXT disponível para download")
        except:
            print("⚠️ Download não disponível neste ambiente")
    except Exception as e:
        print(f"⚠️ Erro ao salvar interações: {e}")

# Função para registrar uma nova interação
def registrar_interacao(tarefa, emocao, resposta_llm):
    interacoes = carregar_interacoes()
    fuso_brasil = pytz.timezone("America/Sao_Paulo")
    nova = {
        "timestamp": datetime.now(fuso_brasil).isoformat(),
        "tarefa": tarefa,
        "emocao": agrupar_emocao(emocao),
        "resposta": resposta_llm
    }
    interacoes.append(nova)
    salvar_interacoes(interacoes)
    print("✅ Interação registrada com sucesso!")
    return nova["resposta"]

# Função para gerar relatório textual
def gerar_relatorio_txt(interacoes):
    caminho = CAMINHO_BASE + "relatorio.txt"
    relatorio = "📊 Relatório Emocional\n"
    relatorio += f"📈 Total de interações: {len(interacoes)}\n\n"
    relatorio += "📋 Histórico de Interações:\n"
    for i in interacoes:
        dt = datetime.fromisoformat(i["timestamp"])
        relatorio += f"- {dt.strftime('%d/%m %H:%M')} | Emoção: {i['emocao']} | Tarefa: {i['tarefa']}\n"
    emocoes = [i["emocao"] for i in interacoes]
    tarefas = [i["tarefa"] for i in interacoes]
    relatorio += "\n📊 Emoções mais frequentes:\n"
    for e, freq in Counter(emocoes).most_common():
        relatorio += f"- {e}: {freq}x\n"
    relatorio += "\n📝 Tarefas mais citadas:\n"
    for t, freq in Counter(tarefas).most_common():
        relatorio += f"- {t}: {freq}x\n"
    relatorio += "\n💡 Sugestões do Agente (últimas 3):\n"
    for r in interacoes[-3:]:
        relatorio += f'- "{r["resposta"]}"\n'
    try:
        with open(caminho, "w") as f:
            f.write(relatorio)
        print("📊 Relatório salvo localmente com sucesso!")
        try:
            files.download(caminho)
            print("📥 Relatório disponível para download")
        except:
            print("⚠️ Download não disponível neste ambiente")
    except Exception as e:
        print(f"⚠️ Erro ao salvar relatório: {e}")
    return relatorio

# Função para gerar relatório visual
def gerar_relatorio_visual(interacoes):
    emocoes = [i["emocao"] for i in interacoes]
    contador = Counter(emocoes)
    plt.figure(figsize=(10, 6))  # Aumentado para melhor legibilidade
    plt.bar(contador.keys(), contador.values())
    plt.title("Frequência de Emoções")
    plt.xlabel("Emoção")
    plt.ylabel("Frequência")
    plt.xticks(rotation=45, ha='right')  # Alinha rótulos à direita para evitar sobreposição
    plt.tight_layout(pad=2.0)  # Aumenta o padding para evitar corte
    caminho = CAMINHO_BASE + "emocoes_plot.png"
    plt.savefig(caminho)
    print("📊 Gráfico salvo localmente em 'emocoes_plot.png'!")
    try:
        from google.colab import files
        files.download(caminho)
        print("📥 Gráfico disponível para download")
    except:
        print("⚠️ Download não disponível neste ambiente")
    plt.close()

In [5]:
# 🔁 Interação com o Agente Emocional Inteligente
pontos_bem_estar = 0  # Variável global para pontos
nivel = "Iniciante"   # Nível inicial

def interagir_com_agente():
    global pontos_bem_estar, nivel
    print("🧠 Agente Emocional Inteligente")
    while True:
        tarefa = input("👉 Qual tarefa você está tentando fazer agora? (ou 'sair' para encerrar)\n> ")
        if tarefa.lower() == 'sair':
            print(f"👋 Até a próxima! Pontos: {pontos_bem_estar} | Nível: {nivel}")
            break

        emocao = input("💬 Como você está se sentindo nesse momento?\n> ")
        if emocao.lower() == 'sair':
            print(f"👋 Até a próxima! Pontos: {pontos_bem_estar} | Nível: {nivel}")
            break

        # Validação de inputs
        if not tarefa.strip():
            print("⚠️ A tarefa não pode estar vazia. Tente novamente.")
            continue
        if not emocao.strip():
            print("⚠️ A emoção não pode estar vazia. Tente novamente.")
            continue
        emocao_normalizada = agrupar_emocao(emocao)
        valid_emotions = list(GRUPOS_EMOCAO.keys()) + [term for sublist in GRUPOS_EMOCAO.values() for term in sublist]
        if emocao_normalizada not in valid_emotions:
            print(f"⚠️ Emoção '{emocao}' não reconhecida. Tente algo como: ansioso, cansado, feliz, triste, motivado.")
            continue

        # 🧾 Criando o prompt com instruções claras para o Gemini
        prompt = f"""
        Você é um agente emocional digital. Sua missão é ajudar o usuário a manter a produtividade sem ignorar o bem-estar emocional.

        Dado:
        - Emoção: {emocao_normalizada}
        - Tarefa: {tarefa}

        Responda de forma breve, empática e prática.
        Inclua:
        1. Uma sugestão simples para ajudar com a tarefa considerando o estado emocional.
        2. Um reforço positivo final.

        Exemplos:
        Usuário: 'Ansioso tentando escrever um relatório'
        Resposta: 'Tente fazer um esboço rápido com os tópicos principais. Isso reduz a ansiedade. Você consegue!'

        Usuário: 'Cansado e precisa revisar um texto'
        Resposta: 'Dê uma pausa de 5 minutos, respire fundo e volte com mais clareza. Seu esforço vale a pena!'

        Agora, gere a resposta para o caso:
        """

        # 🎯 Chamando o modelo Gemini com tratamento de erro
        try:
            model = genai.GenerativeModel("gemini-1.5-flash")
            response = model.generate_content(prompt)
            resposta_gemini = response.text
        except Exception as e:
            print(f"⚠️ Erro ao chamar a API do Gemini: {e}")
            resposta_gemini = gerar_resposta_emocional(emocao_normalizada)

        # 💬 Mostrando a resposta do agente
        print("\n🤖 Sugestão personalizada:")
        print(resposta_gemini)

        # 📝 Registrar a interação com a resposta do Gemini
        registrar_interacao(tarefa, emocao_normalizada, resposta_gemini)

        # Feedback do usuário e gamificação
        print("\n📌 Funcionou? Digite 'sim', 'não' ou 'sair'.")
        feedback = input("> ")
        if feedback.lower() == 'sair':
            print(f"👋 Até a próxima! Pontos: {pontos_bem_estar} | Nível: {nivel}")
            break
        elif feedback.lower() == 'sim':
            pontos_bem_estar += 10
            if pontos_bem_estar >= 30:
                nivel = "Mestre"
            elif pontos_bem_estar >= 20:
                nivel = "Focado"
            elif pontos_bem_estar >= 10:
                nivel = "Intermediário"
            print(f"🎉 Ótimo! +10 pontos. Total: {pontos_bem_estar} | Novo nível: {nivel}")
        elif feedback.lower() == 'não':
            print("🤔 Entendi. Vamos tentar outra sugestão.")
            continue
        else:
            print(f"🎉 Resposta inválida, mas seguimos! Pontos: {pontos_bem_estar} | Nível: {nivel}")

# Executar a interação
interagir_com_agente()

🧠 Agente Emocional Inteligente
👉 Qual tarefa você está tentando fazer agora? (ou 'sair' para encerrar)
> estudando
💬 Como você está se sentindo nesse momento?
> animado

🤖 Sugestão personalizada:
Aproveite essa felicidade!  Coloque uma música animada enquanto estuda para manter o ânimo alto.  Você está brilhando!

💾 Interações salvas localmente com sucesso!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Arquivo JSON disponível para download


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

📥 Arquivo JSON disponível para download


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Arquivo TXT disponível para download
✅ Interação registrada com sucesso!

📌 Funcionou? Digite 'sim', 'não' ou 'sair'.
> sair
👋 Até a próxima! Pontos: 20 | Nível: Focado


In [6]:
# Gerar relatório visual
interacoes = carregar_interacoes()
if interacoes:
    gerar_relatorio_visual(interacoes)
else:
    print("⚠️ Nenhuma interação encontrada para gerar o relatório visual.")

📊 Gráfico salvo localmente em 'emocoes_plot.png'!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Gráfico disponível para download


In [ ]:
# Interface Gradio
!pip install -q gradio

import gradio as gr

def interagir_gradio(tarefa, emocao):
    global pontos_bem_estar, nivel
    print(f"Recebido: tarefa='{tarefa}', emocao='{emocao}'")  # Depuração
    if not tarefa.strip():
        return "⚠️ A tarefa não pode estar vazia. Tente novamente."
    if not emocao.strip():
        return "⚠️ A emoção não pode estar vazia. Tente novamente."
    emocao_normalizada = agrupar_emocao(emocao)
    valid_emotions = list(GRUPOS_EMOCAO.keys()) + [term for sublist in GRUPOS_EMOCAO.values() for term in sublist]
    if emocao_normalizada not in valid_emotions:
        return f"⚠️ Emoção '{emocao}' não reconhecida. Tente algo como: ansioso, cansado, feliz, triste, motivado."

    prompt = f"""
    Você é um agente emocional digital. Sua missão é ajudar o usuário a manter a produtividade sem ignorar o bem-estar emocional.

    Dado:
    - Emoção: {emocao_normalizada}
    - Tarefa: {tarefa}

    Responda de forma breve, empática e prática.
    Inclua:
    1. Uma sugestão simples para ajudar com a tarefa considerando o estado emocional.
    2. Um reforço positivo final.
    """
    try:
        print("Chamando Gemini...")  # Depuração
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt)
        resposta = response.text
        print(f"Resposta Gemini: {resposta}")  # Depuração
    except Exception as e:
        print(f"⚠️ Erro ao chamar a API do Gemini: {e}")  # Depuração
        resposta = gerar_resposta_emocional(emocao_normalizada)

    registrar_interacao(tarefa, emocao_normalizada, resposta)
    resultado = f"🤖 Sugestão: {resposta}\n🎮 Pontos: {pontos_bem_estar} | Nível: {nivel}"
    print(f"Retornando: {resultado}")  # Depuração
    return resultado

def gerar_grafico():
    interacoes = carregar_interacoes()
    if not interacoes:
        return "⚠️ Nenhuma interação encontrada para gerar o gráfico."
    gerar_relatorio_visual(interacoes)
    return CAMINHO_BASE + "emocoes_plot.png"  # Retorna o caminho do arquivo

def sair():
    global pontos_bem_estar, nivel
    return f"👋 Até a próxima! Pontos: {pontos_bem_estar} | Nível: {nivel}"

with gr.Blocks() as demo:
    gr.Markdown("# 🧠 Agente Emocional Inteligente")
    with gr.Row():
        tarefa_input = gr.Textbox(label="👉 Qual tarefa você está tentando fazer?")
        emocao_input = gr.Textbox(label="💬 Como você está se sentindo?")
    with gr.Row():
        submit_btn = gr.Button("Enviar")
        sair_btn = gr.Button("Sair")
    output = gr.Textbox(label="🤖 Resposta do Agente")
    plot_btn = gr.Button("Gerar Gráfico")
    plot_output = gr.Image(label="📊 Gráfico de Emoções")

    submit_btn.click(
        fn=interagir_gradio,
        inputs=[tarefa_input, emocao_input],
        outputs=output
    )
    sair_btn.click(
        fn=sair,
        inputs=None,
        outputs=output
    )
    plot_btn.click(
        fn=gerar_grafico,
        inputs=None,
        outputs=plot_output
    )

demo.launch(debug=True)  # Mantido debug=True para ver erros

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ea20daa5a29a1974b4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Recebido: tarefa='estudando', emocao='feliz'
Chamando Gemini...
Resposta Gemini: Que ótimo que você está feliz enquanto estuda!  Aproveite essa energia positiva fazendo um pequeno intervalo com algo que te deixe ainda mais feliz entre os tópicos de estudo.  Você vai arrasar!

💾 Interações salvas localmente com sucesso!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Arquivo JSON disponível para download


...

<IPython.core.display.Javascript object>

📥 Arquivo TXT disponível para download
✅ Interação registrada com sucesso!
Retornando: 🤖 Sugestão: Que ótimo que você está feliz enquanto estuda!  Aproveite essa energia positiva fazendo um pequeno intervalo com algo que te deixe ainda mais feliz entre os tópicos de estudo.  Você vai arrasar!

🎮 Pontos: 20 | Nível: Focado
📊 Gráfico salvo localmente em 'emocoes_plot.png'!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Gráfico disponível para download
